<a href="https://colab.research.google.com/github/aurill/customer_churn_sprint/blob/main/Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from google.colab import drive

In [72]:
# Installing the neccessary packages & importing libraries
!pip install pyforest
import pyforest

In [61]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [62]:
data= pd.read_excel(r'/content/gdrive/My Drive/Customer Churn/Customer Churn Main.xlsx')

<IPython.core.display.Javascript object>

In [63]:
data.head(6)

,Customer_ID,Age,Contract_Type,Monthly_Charges,Total_Charges,Tenure,Phone_Service,Multiple_Lines,Online_Backup,Churn,Active_Member,Customer Complains
0,68382346,34,1,4219,291111,61,1,0,1,0,1,1
1,56062054,27,1,6541,170066,20,0,0,0,1,1,1
2,49461022,47,0,2792,175896,36,1,1,1,1,1,1
3,68472136,73,1,4852,87336,42,0,1,0,1,1,1
4,55289292,60,0,3439,82536,26,0,1,1,0,1,1
5,55111438,19,1,8732,61124,21,0,1,0,0,0,0


In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer_ID         947 non-null    object
 1   Age                 947 non-null    int64 
 2   Contract_Type       947 non-null    int64 
 3   Monthly_Charges     947 non-null    int64 
 4   Total_Charges       947 non-null    int64 
 5   Tenure              947 non-null    int64 
 6   Phone_Service       947 non-null    int64 
 7   Multiple_Lines      947 non-null    int64 
 8   Online_Backup       947 non-null    int64 
 9   Churn               947 non-null    int64 
 10  Active_Member       947 non-null    int64 
 11  Customer Complains  947 non-null    int64 
dtypes: int64(11), object(1)
memory usage: 88.9+ KB


## **Data Preprocessing**

In [65]:
data.corr()['Churn'].sort_values()

<ipython-input-65-5e91c37ee28d>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()['Churn'].sort_values()


Age                  -0.036481
Contract_Type        -0.026307
Phone_Service        -0.019158
Online_Backup        -0.013572
Monthly_Charges      -0.000162
Total_Charges         0.007913
Multiple_Lines        0.070367
Tenure                0.272682
Active_Member         0.941265
Customer Complains    0.941265
Churn                 1.000000
Name: Churn, dtype: float64

## **Train / Test Split**

In [66]:
# Split data into train and test sets
features = ['Tenure', 'Customer Complains', 'Active_Member']
X_train, X_test, y_train, y_test = train_test_split(data[features], data['Churn'], test_size = 0.25, random_state = 42)
X_train.shape, X_test.shape

<IPython.core.display.Javascript object>

((710, 3), (237, 3))

## **Model Selection**

# **Ridge Classifier with Grid Search**




In [98]:
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score

# Defining the hyperparameter grid
param_grid = {'alpha': [0.1, 1, 10, 100]}

# Creating the Ridge Classifier
ridgeclassifier = RidgeClassifier()

# Performing grid search
grid_search = GridSearchCV(ridgeclassifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Getting the best hyperparameters
best_alpha = grid_search.best_params_['alpha']

# Training the Ridge Classifier with the best hyperparameters
ridgeclassifier = RidgeClassifier(alpha=best_alpha)
ridgeclassifier.fit(X_train, y_train)

# Predicting testing values
y_pred_test = ridgeclassifier.predict(X_test)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred_test) * 100
recall = recall_score(y_test,y_pred) * 100
precision = precision_score(y_test,y_pred) * 100
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[: , 1]) * 100

print(f'Ridge Classifier Accuracy Score : {accuracy:.3f} %')
print(f'Ridge Classifier Recall Score : {recall:.3f} %')
print(f'Ridge Classifier Precision Score : {precision:.3f} %')
print(f'Ridge Classifier roc auc Score : {roc_auc:.3f} %')

Ridge Classifier Accuracy Score : 96.203 %
Ridge Classifier Recall Score : 99.275 %
Ridge Classifier Precision Score : 94.483 %
Ridge Classifier roc auc Score : 94.675 %


# **LightGBM Classifier with Bayesian Optimization**



In [97]:
!pip install scikit-optimize

from skopt import BayesSearchCV
from skopt.space import Real, Integer
import lightgbm as lgb
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score

# Define the search space for hyperparameters
param_space = {
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'max_depth': Integer(1, 32),
    'num_leaves': Integer(2, 256),
    'min_child_samples': Integer(1, 100),
    'subsample': Real(0.1, 1.0, 'uniform'),
    'colsample_bytree': Real(0.1, 1.0, 'uniform'),
    'n_estimators': Integer(50, 1000)
}

# Create a LightGBM Classifier
lgb_classifier = lgb.LGBMClassifier(verbose=-1)

# Use Bayesian optimization with cross-validation to find the best hyperparameters
opt = BayesSearchCV(
    lgb_classifier,
    param_space,
    n_iter=50,  # Number of optimization iterations
    scoring='roc_auc',  # Choose an appropriate metric for your problem
    cv=5,  # Number of cross-validation folds
    n_jobs=-1  # Use all available CPU cores
)
opt.fit(X_train, y_train)

# Get the best hyperparameters
best_params = opt.best_params_

# Train the LightGBM Classifier with the best hyperparameters
best_lgb_classifier = lgb.LGBMClassifier(**best_params, verbose=-1)
best_lgb_classifier.fit(X_train, y_train)

# Evaluate the optimized LightGBM Classifier
y_pred_test = best_lgb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_test) * 100
recall = recall_score(y_test, y_pred_test) * 100
precision = precision_score(y_test, y_pred_test) * 100
roc_auc = roc_auc_score(y_test, best_lgb_classifier.predict_proba(X_test)[:, 1]) * 100

print(f'LightGBM Accuracy Score: {accuracy:.3f} %')
print(f'LightGBM Recall Score: {recall:.3f} %')
print(f'LightGBM Precision Score: {precision:.3f} %')
print(f'LightGBM ROC AUC Score: {roc_auc:.3f} %')


LightGBM Accuracy Score: 96.203 %
LightGBM Recall Score: 99.275 %
LightGBM Precision Score: 94.483 %
LightGBM ROC AUC Score: 94.708 %
